# Player Allocation

Upon completion of evaluation of overall performance for all skaters, each player will be categorized and assigned to their respectful position within the roster of their team. The fundamental statistic used to differentiate top from bottom players, is zone start percentage. There are three zones from where plays can be started: offensive, neutral and defensive. Offensive zone start percentage is calculated by the number of face-offs held in the attacking area divided by the sum of face-offs an individual player was on the ice for. Identically, defensive zone start percentage is computed by the number of face-offs taken in their own territory divided by the total face-offs each player was on the ice for. Skaters who are talented in creating opportunities and producing goals will have a greater percentage of offensive zone starts in comparison to defensive zone starts. Equivalently, players who are skilled in preventing chances and goals being conceded, will have a higher defensive zone start percentage in correlation to offensive zone starts. In other words, top six forwards and top four pairing defensemen will start much of their shifts in the zone their opponent whereas bottom six forwards and bottom pairing defensemen in their own zone.


### purpose of notebook:

a) create two variables: offensive zone start and defensive zone start.

b) sum up the total offensive and defensive zone start per player.

c) categorize forwards into top and bottom six.

d) categorize defensemen into top four and bottom pairing.

e) determine each player's roster depth position

##  import modules

In [122]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

## import data frame

The player evaluation data frame is used for player allocation.

In [123]:
dm = pd.read_csv('plyreval.csv')

## drop unnamed column (irrelevant)

In [124]:
dm = dm.drop('Unnamed: 0', axis=1)

## zone start

With the help of zone variable, offensive, neutral and defensive zone starts will be created.

**zone start variable:** 

- a value of 1 will be assigned if the on-ice event happened in the offensive zone.

- a value of 0 will be assigned if the on-ice event happened in the neutral zone.

- a value of -1 if it happened in the defensive zone of the representative team.

In [125]:
dm['zs'] = np.where(dm['Zone'] == 'O', 1,
                    (np.where(dm['Zone'] == 'D', -1, 0)))

## home and visitor zone start

- visitor team zone start (vzs)

If team code of event is the same as visitor team, the visitor zone start variable will be assigned identical value to zone start. If not, it will be assigned the opposite (negative) value of zone start. 

In [126]:
dm['vzs'] = np.where(dm['TeamCode'] == dm['VTeamCode'], dm['zs'], -dm['zs'] )

- home team zone start (hzs) 

If team code of event is the same as home team, the home team will be assigned identical value to zone start. If not, it will be assigned the opposite (negative) value of zone start. 

In [127]:
dm['hzs'] = np.where(dm['TeamCode'] == dm['HTeamCode'], dm['zs'], -dm['zs'] )

## assign zone start to players

The value of zone start is assigned to all players that were on ice, a total of 12 players (6 per team). The overall zone start variable of each player is the total (sum) of events they participated in. 

### a) overall zone start of each player from the visitor team in all (6) positions 

Group data frame by season, visitor team code and visitor player position to seperate players that play in the same position. 

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer1"

In [128]:
dm['zvp1'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer1'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer2"

In [129]:
dm['zvp2'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer2'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer3"

In [130]:
dm['zvp3'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer3'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer4"

In [131]:
dm['zvp4'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer4'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer5"

In [132]:
dm['zvp5'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer5'])['vzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "VPlayer6"

In [133]:
dm['zvp6'] = dm.groupby(['Season', 'VTeamCode', 'VPlayer6'])['vzs'].transform('sum')

### b) overall zone start of each player from the home team in all (6) positions

Group data frame by season, home team code and home player position to seperate players that play in the same position.

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer1"

In [134]:
dm['zhp1'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer1'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer2"

In [135]:
dm['zhp2'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer2'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer3"

In [136]:
dm['zhp3'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer3'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer4"

In [137]:
dm['zhp4'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer4'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer5"

In [138]:
dm['zhp5'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer5'])['hzs'].transform('sum')

- create variable that sums up the overall zone start of each player from the visitor team that is listed as "HPlayer6"

In [139]:
dm['zhp6'] = dm.groupby(['Season', 'HTeamCode', 'HPlayer6'])['hzs'].transform('sum')

## players that play in multiple positions

Throughout the duration of a game, a player may change position. As mentioned above, the overall impact of a given player is the total (sum) of events he participated in. To properly measure each player's contribution, a cross examination per position must be applied.

- **a) cross examine each position for visitor team:**

**visitor player 1**

- if visitor player in position 1 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 1 and 2.

In [140]:
dm['tzvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer2']), dm['zvp1'] + dm['zvp2'], dm['zvp1'])
   

- if visitor player in position 1 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 1 and 3.

In [141]:
dm['tzvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer3']), dm['zvp1'] + dm['zvp3'], dm['zvp1'])
   

- if visitor player in position 1 played any games in position 4 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 1 and 4.

In [142]:
dm['tzvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer4']), dm['zvp1'] + dm['zvp4'], dm['zvp1'])
   

- if visitor player in position 1 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 1 and 5.

In [143]:
dm['tzvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer5']), dm['zvp1'] + dm['zvp5'], dm['zvp1'])
   

- if visitor player in position 1 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of zone starts for position 1 and 6.

In [144]:
dm['tzvp1'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer1'] == dm['VPlayer6']), dm['zvp1'] + dm['zvp6'], dm['zvp1'])
   

**visitor player 2**

- if visitor player in position 2 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 1 and 2.

In [145]:
dm['tzvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer1']), dm['zvp2'] + dm['zvp1'], dm['zvp2'])
   

- if visitor player in position 2 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 2 and 3.

In [146]:
dm['tzvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer3']), dm['zvp2'] + dm['zvp3'], dm['zvp2'])
   

- if visitor player in position 2 played any games in position 4 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 2 and 4.

In [147]:
dm['tzvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer4']), dm['zvp2'] + dm['zvp4'], dm['zvp2'])
   

- if visitor player in position 2 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 2 and 5.

In [148]:
dm['tzvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer5']), dm['zvp2'] + dm['zvp5'], dm['zvp2'])
   

- if visitor player in position 2 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of zone starts for position 2 and 6.

In [149]:
dm['tzvp2'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer2'] == dm['VPlayer6']), dm['zvp2'] + dm['zvp6'], dm['zvp2'])
   

** visitor player 3**

- if visitor player in position 3 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 1 and 3.

In [150]:
dm['tzvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer1']), dm['zvp3'] + dm['zvp1'], dm['zvp3'])
   

- if visitor player in position 3 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 3 and 2.

In [151]:
dm['tzvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer2']), dm['zvp3'] + dm['zvp2'], dm['zvp3'])
   

- if visitor player in position 3 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 3 and 4.

In [152]:
dm['tzvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer4']), dm['zvp3'] + dm['zvp4'], dm['zvp3'])
   

- if visitor player in position 3 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 3 and 5.

In [153]:
dm['tzvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer5']), dm['zvp3'] + dm['zvp5'], dm['zvp3'])
   

- if visitor player in position 3 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of zone starts for position 3 and 6.

In [154]:
dm['tzvp3'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer3'] == dm['VPlayer6']), dm['zvp3'] + dm['zvp6'], dm['zvp3'])
   

** visitor player 4**

- if visitor player in position 4 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 4 and 1.

In [155]:
dm['tzvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer1']), dm['zvp4'] + dm['zvp1'], dm['zvp4'])
   

- if visitor player in position 4 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 4 and 2.

In [156]:
dm['tzvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer2']), dm['zvp4'] + dm['zvp2'], dm['zvp4'])
   

- if visitor player in position 4 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 4 and 3.

In [157]:
dm['tzvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer3']), dm['zvp4'] + dm['zvp3'], dm['zvp4'])
   

- if visitor player in position 4 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 4 and 5.

In [158]:
dm['tzvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer5']), dm['zvp4'] + dm['zvp5'], dm['zvp4'])
   

- if visitor player in position 4 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of zone starts for position 4 and 6.

In [159]:
dm['tzvp4'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer4'] == dm['VPlayer6']), dm['zvp4'] + dm['zvp6'], dm['zvp4'])
   

** visitor player 5**

- if visitor player in position 5 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 5 and 1.

In [160]:
dm['tzvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer1']), dm['zvp5'] + dm['zvp1'], dm['zvp5'])
   

- if visitor player in position 5 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 5 and 2.

In [161]:
dm['tzvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer2']), dm['zvp5'] + dm['zvp2'], dm['zvp5'])
   

- if visitor player in position 5 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 5 and 3.

In [162]:
dm['tzvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer3']), dm['zvp5'] + dm['zvp3'], dm['zvp5'])
   

- if visitor player in position 5 played any games in position 4 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 5 and 4.

In [163]:
dm['tzvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer4']), dm['zvp5'] + dm['zvp4'], dm['zvp5'])
   

- if visitor player in position 5 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sum of zone starts for position 5 and 6.

In [164]:
dm['tzvp5'] = np.where((dm['VTeamCode'] == dm['VTeamCode']) & (dm['VPlayer5'] == dm['VPlayer6']), dm['zvp5'] + dm['zvp6'], dm['zvp5'])
   

**visitor player 6**

Position 6 is the goaltender position. A goalie doesn't play in any other position, therefore :

In [165]:
dm['tzvp6'] = dm['zvp6']

- **b) cross examine each position for home team:**

**home position 1**

- if home player in position 1 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sum of zone starts for position 1 and 2.

In [166]:
dm['tzhp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer2']), dm['zhp1'] + dm['zhp2'], dm['zhp1'])
   

- if home player in position 1 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 1 and 3.

In [167]:
dm['tzhp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer3']), dm['zhp1'] + dm['zhp3'], dm['zhp1'])
   

- if home player in position 1 played any games in position 4 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 1 and 4.

In [168]:
dm['tzhp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer4']), dm['zhp1'] + dm['zhp4'], dm['zhp1'])
   

- if home player in position 1 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 1 and 5.

In [169]:
dm['tzhp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer5']), dm['zhp1'] + dm['zhp5'], dm['zhp1'])
   

- if visitor player in position 1 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sume of zone starts for position 1 and 6.

In [170]:
dm['tzhp1'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer1'] == dm['HPlayer6']), dm['zhp1'] + dm['zhp6'], dm['zhp1'])
   

** home player 2**

- if home player in position 2 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 2 and 1.

In [171]:
dm['tzhp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer1']), dm['zhp2'] + dm['zhp1'], dm['zhp2'])
   

- if home player in position 2 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 2 and 3.

In [172]:
dm['tzhp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer3']), dm['zhp2'] + dm['zhp3'], dm['zhp2'])
   

- if home player in position 2 played any games in position 4 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 2 and 4.

In [173]:
dm['tzhp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer4']), dm['zhp2'] + dm['zhp4'], dm['zhp2'])
   

- if home player in position 2 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 2 and 5.

In [174]:
dm['tzhp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer5']), dm['zhp2'] + dm['zhp5'], dm['zhp2'])
   

- if visitor player in position 2 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sume of zone starts for position 2 and 6.

In [175]:
dm['tzhp2'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer2'] == dm['HPlayer6']), dm['zhp2'] + dm['zhp6'], dm['zhp2'])
   

** home player 3**

- if home player in position 3 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 3 and 1.

In [176]:
dm['tzhp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer1']), dm['zhp3'] + dm['zhp1'], dm['zhp3'])
   

- if home player in position 3 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 3 and 2.

In [177]:
dm['tzhp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer2']), dm['zhp3'] + dm['zhp2'], dm['zhp3'])
   

- if home player in position 3 played any games in position 4 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 3 and 4.

In [178]:
dm['tzhp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer4']), dm['zhp3'] + dm['zhp4'], dm['zhp3'])
   

- if home player in position 3 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 3 and 5.

In [179]:
dm['tzhp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer5']), dm['zhp3'] + dm['zhp5'], dm['zhp3'])
   

- if visitor player in position 3 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sume of zone starts for position 3 and 6.

In [180]:
dm['tzhp3'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer3'] == dm['HPlayer6']), dm['zhp3'] + dm['zhp6'], dm['zhp3'])
   

** home player 4**

- if home player in position 4 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 4 and 1.

In [181]:
dm['tzhp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer1']), dm['zhp4'] + dm['zhp1'], dm['zhp4'])
   

- if home player in position 4 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 4 and 2.

In [182]:
dm['tzhp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer2']), dm['zhp4'] + dm['zhp2'], dm['zhp4'])
   

- if home player in position 4 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 4 and 3.

In [183]:
dm['tzhp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer3']), dm['zhp4'] + dm['zhp3'], dm['zhp4'])
   

- if home player in position 4 played any games in position 5 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 4 and 5.

In [184]:
dm['tzhp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer5']), dm['zhp4'] + dm['zhp5'], dm['zhp4'])
   

- if visitor player in position 4 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sume of zone starts for position 4 and 6.

In [185]:
dm['tzhp4'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer4'] == dm['HPlayer6']), dm['zhp4'] + dm['zhp6'], dm['zhp4'])
   

**home player 5**

- if home player in position 5 played any games in position 1 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 5 and 1.

In [186]:
dm['tzhp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer1']), dm['zhp5'] + dm['zhp1'], dm['zhp5'])
   

- if home player in position 5 played any games in position 2 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 5 and 2.

In [187]:
dm['tzhp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer2']), dm['zhp5'] + dm['zhp2'], dm['zhp5'])
   

- if home player in position 5 played any games in position 3 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 5 and 3.

In [188]:
dm['tzhp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer3']), dm['zhp5'] + dm['zhp3'], dm['zhp5'])
   

- if home player in position 5 played any games in position 4 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. It is the sume of zone starts for position 5 and 4.

In [189]:
dm['tzhp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer4']), dm['zhp5'] + dm['zhp4'], dm['zhp5'])
   

- if visitor player in position 5 played any games in position 6 throughtout the season, the total of zone starts he was on the ice for are calculated with the use of **np.where**. A player might play in position 6 (goaltender position) when his team is trailing in the final minutes of a game and they decide to pull the goaltender to add an additonal skater. It is the sume of zone starts for position 5 and 6.

In [190]:
dm['tzhp5'] = np.where((dm['HTeamCode'] == dm['HTeamCode']) & (dm['HPlayer5'] == dm['HPlayer6']), dm['zhp5'] + dm['zhp6'], dm['zhp5'])
   

** home player 6**

Position 6 is the goaltender position. A goalie doesn't play in any other position, therefore :

In [191]:
dm['tzhp6'] = dm['zhp6']

## overall zone starts

Zone starts of each player has been calculated only for his team being home or away for the season, since home zone start value and visitor zone start value were used. The **total zone starts** of each player is the total of zone starts he participated for a whole season. Thus, the sum of both home and away zone starts.

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 1.

In [192]:
dm['zplyr1'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer1'] == dm['VPlayer1']), dm['tzhp1'] + dm['tzvp1'], dm['tzhp1'])
   

In [193]:
dm['zplyr1'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer1'] == dm['HPlayer1']), dm['tzvp1'] + dm['tzhp1'], dm['tzvp1'])
   

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 2.

In [194]:
dm['zplyr2'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer2'] == dm['VPlayer2']), dm['tzhp2'] + dm['tzvp2'], dm['tzhp2'])
   

In [195]:
dm['zplyr2'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer2'] == dm['HPlayer2']), dm['tzvp2'] + dm['tzhp2'], dm['tzvp2'])
   

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 3.

In [196]:
dm['zplyr3'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer3'] == dm['VPlayer3']), dm['tzhp3'] + dm['tzvp3'], dm['tzhp3'])
   

In [197]:
dm['zplyr3'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer3'] == dm['HPlayer3']), dm['tzvp3'] + dm['tzhp3'], dm['tzvp3'])
   

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 4.

In [198]:
dm['zplyr4'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer4'] == dm['VPlayer4']), dm['tzhp4'] + dm['tzvp4'], dm['tzhp4'])
   

In [199]:
dm['zplyr4'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer4'] == dm['HPlayer4']), dm['tzvp4'] + dm['tzhp4'], dm['tzvp4'])
   

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 5.

In [200]:
dm['zplyr5'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer5'] == dm['VPlayer5']), dm['tzhp5'] + dm['tzvp5'], dm['tzhp5'])
   

In [201]:
dm['zplyr5'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer5'] == dm['HPlayer5']), dm['tzvp5'] + dm['tzhp5'], dm['tzvp5'])
   

- create a variable will add up the home zone start value and away zone start value for all players of a given team, that played in position 6.

In [202]:
dm['zplyr6'] = np.where((dm['HTeamCode'] == dm['VTeamCode']) & (dm['HPlayer6'] == dm['VPlayer6']), dm['tzhp6'] + dm['tzvp6'], dm['tzhp6'])
   

In [203]:
dm['zplyr6'] = np.where((dm['VTeamCode'] == dm['HTeamCode']) & (dm['VPlayer6'] == dm['HPlayer6']), dm['tzvp6'] + dm['tzhp6'], dm['tzvp6'])
   

## allocate players per position to forward lines and defensive pairings

- If the zone start variable of a player for a given team is the highest in that specific position, it indicates that he participated in the most offensive zone starts. These skaters will be identified as **first line forwards and top defenisive pairing**. 

- If the zone start variable of a player is the second highest in that specific position, it indicates that the given skater participated in the second most offensive zone starts. These skaters will be identified as **second line forwards and second defenisive pairing**. 

- If the zone start variable of a player is the third highest in that specific position, it indicates that the given skater participated in the third most offensive zone starts. These skaters will be identified as **third line forwards and bottom defenisive pairing**. 

- If the zone start variable of a player is the lowest in that specific position, it indicates that the given skater participated in the least offensive zone starts. These skaters will be identified as **fourth line forwards**. 

### a) visitor team

- generate a variable that will allocate all players in position 1 to their respectful line. Position 1 is the **centre** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [204]:
conditions = [ dm['tzvp1'] > 2, (dm['tzvp1'] <= 2) & (dm['tzvp1']> 0), (dm['tzvp1'] <= 0) & (dm['tzvp1']> -2), dm['tzvp1'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['vc'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 2 to their respectful line. Position 2 is the **right wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [205]:
conditions = [ dm['tzvp2'] > 2, (dm['tzvp2'] <= 2) & (dm['tzvp2']> 0), (dm['tzvp2'] <= 0) & (dm['tzvp2']> -2), dm['tzvp2'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['vrw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 3 to their respectful line. Position 3 is the **left wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [206]:
conditions = [ dm['tzvp3'] > 2, (dm['tzvp3'] <= 2) & (dm['tzvp3']> 0), (dm['tzvp3'] <= 0) & (dm['tzvp3']> -2), dm['tzvp3'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['vlw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 4 to their respectful line. Position 4 is the **right defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.   

In [207]:
conditions = [ dm['tzvp4'] > 3, (dm['tzvp4'] <= 3) & (dm['tzvp4']> 0), dm['tzvp4'] <= 0 ]
choices     = [ 1, 2, 3]

dm['vdr'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 5 to their respectful line. Position 5 is the **left defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.  

In [208]:
conditions = [ dm['tzvp5'] > 3, (dm['tzvp5'] <= 3) & (dm['tzvp5']> 0), dm['tzvp5'] <= 0 ]
choices     = [ 1, 2, 3]

dm['vdl'] = np.select(conditions, choices, default=np.nan)

- Position 6 is the goaltender position. No conditions necessary 

In [209]:
dm['vg'] = 1

### b) home team

- generate a variable that will allocate all players in position 5 to their respectful line. Position 5 is the **left defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**. 

In [210]:
conditions = [ dm['tzhp1'] > 2, (dm['tzhp1'] <= 2) & (dm['tzhp1']> 0), (dm['tzhp1'] <= 0) & (dm['tzhp1']> -2), dm['tzhp1'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['hc'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 2 to their respectful line. Position 2 is the **right wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [211]:
conditions = [ dm['tzhp2'] > 2, (dm['tzhp2'] <= 2) & (dm['tzhp2']> 0), (dm['tzhp2'] <= 0) & (dm['tzhp2']> -2), dm['tzhp2'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['hrw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 3 to their respectful line. Position 3 is the **left wing** position of forward lines. If **total zone start > 2**, player is assinged to the **top line**. If **total zone start between 2 and 0**, player is assinged to the **2nd line**. If **total zone start between 0 and -2**, player is assigned to the **3rd line**. Finally, if **total zone start less than -2**, player assigned to **4th line**.  

In [212]:
conditions = [ dm['tzhp3'] > 2, (dm['tzhp3'] <= 2) & (dm['tzhp3']> 0), (dm['tzhp3'] <= 0) & (dm['tzhp3']> -2), dm['tzhp3'] <= -2 ]
choices     = [ 1, 2, 3, 4 ]

dm['hlw'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 4 to their respectful line. Position 4 is the **right defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.   

In [213]:
conditions = [ dm['tzhp4'] > 3, (dm['tzhp4'] <= 3) & (dm['tzhp4']> 0), dm['tzhp4'] <= 0 ]
choices     = [ 1, 2, 3]

dm['hdr'] = np.select(conditions, choices, default=np.nan)

- generate a variable that will allocate all players in position 5 to their respectful line. Position 5 is the **left defense** position. If **total zone start > 3**, player is assinged to the **top defense pairing**. If **total zone start between 3 and 0**, player is assinged to the **2nd defense pairng**. Finally, if **total zone start less than 0**, player assigned to **3rd defense pairing**.  

In [214]:
conditions = [ dm['tzhp5'] > 3, (dm['tzhp5'] <= 3) & (dm['tzhp5']> 0), dm['tzhp5'] <= 0 ]
choices     = [ 1, 2, 3]

dm['hdl'] = np.select(conditions, choices, default=np.nan)

- Position 6 is the goaltender position. No conditions necessary 

In [215]:
dm['vg'] = 1